In [1]:
import tensorflow as tf
import numpy as np
import yaml
from abc import abstractmethod

tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
# module definitions
class TensorDict(dict):
    """
    Class that contains a dictionary of tensors
    """
    def last(self):
        return self.get(list(self.keys())[-1])
    
    def get_many(self, t_list):
        return {x: self.get(x) for x in t_list}

activation_dict = {"relu": tf.nn.relu, None: None, "None": None}

def create_conv_layer(tensor_in, spec, name, mode=tf.estimator.ModeKeys.TRAIN):
    return {name: tf.layers.conv2d(inputs=tensor_in, 
                                   filters=spec['filters'],
                                   kernel_size=spec['kernel_size'],
                                   padding=spec['padding'],
                                   activation=activation_dict[spec['activation']],
                                   name=name)}

def create_maxpool_layer(tensor_in, spec, name, mode=tf.estimator.ModeKeys.TRAIN):
    return {name: tf.layers.max_pooling2d(inputs=tensor_in,
                                          pool_size=spec['pool_size'],
                                          strides=spec['strides'],
                                          name=name)}

def create_dense_layer(tensor_in, spec, name, mode=tf.estimator.ModeKeys.TRAIN):
    # creates an extra layer that must be flattened as well
    out = {}
    out[name+'_flatten'] = tf.reshape(tensor_in, [-1, np.prod(tensor_in.shape[1:4])])
    out[name] = tf.layers.dense(inputs=out[name+'_flatten'],
                                units=spec['units'],
                                activation=activation_dict[spec['activation']],
                                name=name)
    return out
                      
def create_dropout_layer(tensor_in, spec, name, mode=tf.estimator.ModeKeys.TRAIN):
    return {name: tf.layers.dropout(inputs=tensor_in, rate=spec['rate'], training = mode == tf.estimator.ModeKeys.TRAIN) }

l_dict = {'CONV':create_conv_layer, 'POOL':create_maxpool_layer, 'FC': create_dense_layer, 'DROP': create_dropout_layer}

def tensorboard_summaries(conv_layers=None, dense_layers=None):
    if conv_layers is not None:
        for cl_name, cl in conv_layers.items():
            shape = cl.shape.as_list()
            for f in range(shape[3]):
                tf.summary.image("{}/filter/{}".format(cl_name, f), tf.reshape(cl[:, :, :, f], [tf.shape(cl)[0], shape[1], shape[2], 1]))  
    if dense_layers is not None:
        for dl_name, dl in dense_layers.items():
           tf.summary.histogram("{}/weights".format(dl_name), dl) 

        
class confEstimator(tf.estimator.Estimator):
    """
    Class that allows building a certain neural-net based on a configuration YAML file. It must be overcharged.
    """
            
    def __init__(self, conf_file, model_dir):
        self.conf_file = conf_file
        with open(conf_file) as f:
            self.conf = yaml.load(f)
        super(confEstimator, self).__init__(self.conf_model_fn, model_dir=model_dir)
        
    @abstractmethod
    def conf_model_fn(self, features, labels, model):
        pass
    
    def _get_tensor_dict(self, data_in, mode):
        tensor_dict = TensorDict()  # will contain all the information on the layers and tensors
        num_channels = 1 if len(data_in.shape)==3 else data_in.shape[3]
        tensor_dict['input_layer'] = tf.reshape(data_in, [-1, data_in.shape[1], data_in.shape[2], num_channels]) 
        for l_name, l_def in self.conf['layers'].items():
            tensor_dict.update(l_dict[l_def['type']](tensor_dict.last(), l_def['specs'], l_name, mode))
        return tensor_dict
    
    def get_layers_of_type(self, l_type):
        return [l_name for l_name, l_def in self.conf['layers'].items() if l_def['type']==l_type]
    

class confClassifierEstimator(confEstimator):
    """
    Class that allows for a image classification based on a convolutional neural-net which architecture is on a YAML file
    """
    def conf_model_fn(self, features, labels, mode):

        # tensor_dict
        td =  self._get_tensor_dict(features, mode)
        
        # tensorboard summary for weights and variables in the layers   
        tensorboard_summaries(conv_layers=td.get_many(self.get_layers_of_type('CONV')), 
                             dense_layers=td.get_many(self.get_layers_of_type('FC')))

        # predictions
        predictions = {
            "classes": tf.argmax(input=td.last(), axis=1), 
            "probabilities": tf.nn.softmax(td.last(), name="softmax_tensor")
        }

        if mode == tf.estimator.ModeKeys.PREDICT:
            # wrap predictions into a class and return EstimatorSpec object
            return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

        # one-hot encoding
        onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=td.last().shape[1])

        # minimize on cross-entropy
        loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=td.last())  # loss is a scalar tensor

        if mode == tf.estimator.ModeKeys.TRAIN:
            optimizer = tf.train.GradientDescentOptimizer(learning_rate=self.conf['learning_rate'])
            train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
            return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)


        eval_metric_ops = {
            "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"]),
            "precision": tf.metrics.precision(labels=labels, predictions=predictions["classes"]),
            "recall": tf.metrics.recall(labels=labels, predictions=predictions["classes"])
        }
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [3]:
import datetime


d = datetime.datetime.now()

# load data
mnist = tf.keras.datasets.mnist.load_data(path='mnist.pz')  ## (x_train, y_train), (x_test, y_test)
train_features = np.asarray(mnist[0][0], dtype=np.float32)
train_labels = np.asarray(mnist[0][1], dtype=np.int32)
eval_features = np.asarray(mnist[1][0], dtype=np.float32)
eval_labels = np.asarray(mnist[1][1], dtype=np.int32)

# create the estimator
mnist_classifier = confClassifierEstimator('example_config.yml', 
                                              model_dir="/mnt/data/tensorboard/mnist_conf_convnet_model_{}".format(d.strftime("%Y%M%dT%H%m%S")))

# setup a logging hook
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

# train
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=train_features, 
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(input_fn=train_input_fn, steps=20000) #, hooks=[logging_hook])

# evaluate
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=eval_features,
    y=eval_labels,
    num_epochs=1, 
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/mnt/data/tensorboard/mnist_conf_convnet_model_20184931T090535', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa1cf8c9208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /mnt/data/t

In [5]:
train_features.shape

(60000, 28, 28)

In [6]:
train_labels.shape

(60000,)